**Thank you for visiting. 
This time, I created another random forest for basketball prediction.**

However, as I didn't know how to create train and test data, I copied some codes from https://www.kaggle.com/pixyz0130/eng-mmlm-women-s-lgbm-baseline.
Then, I created my own code to predict the outcome of the basketball tornament, and I succeeded in improving the score.

Okay. Let's get started!!

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
class config:    
    
    SEED = 42
    FOLD_TO_RUN = 0
    FOLDS = 5
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    
    model_type = 'lgb_regression'  
    EPOCHS = 20000
    LR = 2e-4
    message='baseline'
    TARGET = 'target'
    
    TEST = False

In [ ]:
if config.TEST:
    config.EPOCHS = 100

In [ ]:
!ls ../input/womens-march-mania-2022/WDataFiles_Stage1

In [ ]:
df_TDresults = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneyDetailedResults.csv")
df_RSDresults = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WRegularSeasonDetailedResults.csv")
df_TDresults.head()

In [ ]:
df_TDresults = pd.merge(df_RSDresults,df_TDresults,on = df_TDresults.columns.tolist(),how = "outer")
len(df_TDresults)

In [ ]:
box_col = ["FGM","FGA","FGM3","FTM","FTA","OR","DR","Ast","TO","Stl","Blk","PF"]
df_boxW = df_TDresults[["Season","WTeamID"]+["W" + col for col in box_col]]
df_boxL = df_TDresults[["Season","LTeamID"]+["L" + col for col in box_col]]
df_boxW = df_boxW.rename(columns={"WTeamID":"TeamID"})
df_boxW = df_boxW.rename(columns={("W"+ col):col for col in box_col})
df_boxL = df_boxL.rename(columns={"LTeamID":"TeamID"})
df_boxL = df_boxL.rename(columns={("L"+ col):col for col in box_col})
df_box = pd.merge(df_boxW,df_boxL,on = ["Season","TeamID"]+box_col,how="outer")
df_box = df_box.groupby(["Season","TeamID"])[box_col].agg(np.mean).reset_index()
df_box.head()

In [ ]:
df_TDresults2 = df_TDresults
df_TDresults = df_TDresults.rename(columns={"WTeamID":"Team1ID","LTeamID":"Team2ID","WScore":"T1Score","LScore":"T2Score"})
df_TDresults = df_TDresults.rename(columns={f"W{col}":f"T1{col}" for col in box_col})
df_TDresults = df_TDresults.rename(columns={f"L{col}":f"T2{col}" for col in box_col})
df_TDresults2 = df_TDresults2.rename(columns={"WTeamID":"Team2ID","LTeamID":"Team1ID","WScore":"T2Score","LScore":"T1Score"})

In [ ]:
features = ["Season","Team1ID","Team2ID","T1Score","T2Score",'target']
df_TDresults['target'] = 1.0
df_TDresults2['target'] = 0.0
train = pd.merge(df_TDresults,df_TDresults2,on = features,how="outer")
train = train[features]
train.head()

In [ ]:
box_T1 = df_box.copy()
box_T2 = df_box.copy()
box_T1.columns = ['Season','Team1ID'] + ["T1"+col+"_mean" for col in box_col]
box_T2.columns = ['Season','Team2ID'] + ["T2"+col+"_mean" for col in box_col]
train = pd.merge(train,box_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,box_T2,on = ["Season","Team2ID"],how = "left")

In [ ]:
df_seeds = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WNCAATourneySeeds.csv")
df_seeds.head()

In [ ]:
df_seeds["seed"] = df_seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T2 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','Team1ID','T1_seed']
seeds_T2.columns = ['Season','Team2ID','T2_seed']
train = pd.merge(train,seeds_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,seeds_T2,on = ["Season","Team2ID"],how = "left")
train["seeddiff"] = train["T1_seed"] - train["T2_seed"]

In [ ]:
features = train.drop(columns = ["target","T1Score","T2Score"]).columns
train.head()

In [ ]:
test = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WSampleSubmissionStage1.csv")
test.head()

In [ ]:
test["Season"] = test['ID'].apply(lambda x: int(x[0:4]))
test["Team1ID"] = test['ID'].apply(lambda x: int(x[5:9]))
test["Team2ID"] = test['ID'].apply(lambda x: int(x[10:14]))
test.head()

In [ ]:
test = pd.merge(test,box_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,box_T2,on = ["Season","Team2ID"],how = "left")
test.head()

In [ ]:
test = pd.merge(test,seeds_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,seeds_T2,on = ["Season","Team2ID"],how = "left")
test["seeddiff"] = test["T1_seed"] - test["T2_seed"]

In [ ]:
test = test.drop(columns = ['ID','Pred'])
test.head()

**from now on, I create random forest model on my own.**

First, check the missing values.

In [ ]:
train.isnull().sum()

Hmm... Delete it!

In [ ]:
train = train.drop(columns=['T1_seed','T2_seed','seeddiff','T1Score','T2Score'])

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
test = test.drop(columns = ['T1_seed','T2_seed','seeddiff'])

In [ ]:
test.isnull().sum()

Then, divide train and test data.

In [ ]:
y_train = train['target']
X_train = train.drop('target', axis=1)
X_test = test

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rg = RandomForestRegressor(n_jobs=-1, random_state=2525)

In [ ]:
rg.fit(X_train, y_train)

In [ ]:
y_pred = rg.predict(X_test)

Finally, create a csv file!!

In [ ]:
sub = pd.read_csv("../input/womens-march-mania-2022/WDataFiles_Stage1/WSampleSubmissionStage1.csv")
sub['Pred'] = list(map(str, y_pred))
sub.to_csv("submission.csv", index=False)

**That's all. Thank you for reading!!
I appreciate your feedback.**